In [ ]:
import numpy as np
import pandas as pd

import plotly.graph_objects as go
from plotly import express as px
from plotly.offline import init_notebook_mode, iplot

import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS

In [ ]:
init_notebook_mode(connected=True)

# set max column width to 500
pd.set_option("display.max_colwidth", 500)

In [ ]:
DATA_PATH = "/kaggle/input/lmsys-chatbot-arena/train.csv"

# Data analysis

In [ ]:
df = pd.read_csv(DATA_PATH)
df.head()

In [ ]:
df.info()

# Models performance analysis

In [ ]:
model_usage = pd.concat([df["model_a"], df["model_b"]]).value_counts()

# How many unique models are there?
len(model_usage.keys())

In [ ]:
# plot the model usage
fig = px.bar(
    x=model_usage.index,
    y=model_usage.values,
    labels={"x": "Model", "y": "Usage times"},
    title="Model Usage Distribution",
)

fig.update_layout(width=1200, height=700)

iplot(fig)

In [ ]:
# count the number of times each pair of models are compared, regardless of the order
compared_models_count = (
    pd.DataFrame(
        np.sort(df[["model_a", "model_b"]].values, axis=1),
        columns=["model_a", "model_b"],
    )
    .value_counts()
    .reset_index(name="counts")
)

len(compared_models_count)

In [ ]:
top_compared_models = compared_models_count.head(20)

fig = px.bar(
    x=top_compared_models["model_a"] + " vs " + top_compared_models["model_b"],
    y=top_compared_models["counts"],
    labels={"x": "Model Comparison", "y": "Comparison times"},
    title="Top 20 Model Comparison Distribution",
)

iplot(fig)

In [ ]:
# there are 1275 combinations of compared models. How often given numbers of comparisons occur?

compared_models_count["counts"].value_counts()

In [ ]:
# bin the comparison count distribution to 10 bins
comparision_count_distribution_bins = pd.cut(
    compared_models_count["counts"],
    bins=[
        0,
        5,
        10,
        20,
        30,
        40,
        50,
        60,
        70,
        80,
        90,
        100,
        compared_models_count["counts"].max(),
    ],
    precision=0,
    retbins=False,
)

comparision_count_distribution_bins.value_counts(normalize=True).sort_index()

**Which models performs the best and the worst?**

In [ ]:
model_scores_part1 = df.groupby("model_a")["winner_model_a"].sum()
model_scores_part2 = df.groupby("model_b")["winner_model_b"].sum()

model_scores = model_scores_part1.add(model_scores_part2, fill_value=0)
model_scores = model_scores / (
    df["model_a"].value_counts() + df["model_b"].value_counts()
)
model_scores = model_scores.sort_values(ascending=False)

In [ ]:
# top 10 best models
fig = px.bar(
    x=model_scores.head(10).index,
    y=model_scores.head(10).values,
    labels={"x": "Model", "y": "Score"},
    title="Top 10 best-rated models",
)

iplot(fig)

In [ ]:
# top 10 worst models
fig = px.bar(
    x=model_scores.tail(10).index,
    y=model_scores.tail(10).values,
    labels={"x": "Model", "y": "Score"},
    title="Top 10 worst-rated models",
)

iplot(fig)

In [ ]:
tie_df = df.query("winner_tie == 1")
(tie_df["winner_model_a"] + tie_df["winner_model_b"]).value_counts()

**How frequent is tie?**

In [ ]:
f"{(tie_df.shape[0] / df.shape[0]) * 100:.1f}%"

# Positive and negative responses analysis

In [ ]:
# exclude tied examples
df_no_ties = df.query("winner_tie == 0")

loosing_responses = df_no_ties.apply(
    lambda x: (x["response_a"] if x["winner_model_a"] == 0 else x["response_b"]),
    axis=1,
)

loosing_responses

In [ ]:
winning_responses = df_no_ties.apply(
    lambda x: (x["response_a"] if x["winner_model_a"] == 1 else x["response_b"]),
    axis=1,
)

In [ ]:
# loosing responses must be cleaned
loosing_responses = loosing_responses.str.strip("[]")
loosing_responses = loosing_responses.str.strip('"')

winning_responses = winning_responses.str.strip("[]")
winning_responses = winning_responses.str.strip('"')

In [ ]:
df_no_ties["winning_response"] = winning_responses
df_no_ties["loosing_response"] = loosing_responses

In [ ]:
texts = [
    "I do not feel comfortable",
    "I'm sorry, but",
    "I am sorry, but",
    "I apologize, but",
    "Unfortunately I",
    "I do not have enough",
    "I'm just an AI",
    "I'm an AI and",
    "I'm afraid",
    "I can't",
]

unprecise_responses = []
for _, row in df_no_ties.iterrows():
    if row["loosing_response"].startswith(tuple(texts)):
        if row["winning_response"].startswith(tuple(texts)):
            unprecise_responses.append("both")
        else:
            unprecise_responses.append("loosing")
    elif row["winning_response"].startswith(tuple(texts)):
        unprecise_responses.append("winning")
    else:
        continue

pd.Series(unprecise_responses).value_counts(normalize=True)

In [ ]:
import os
os.environ["KERAS_BACKEND"] = "jax"  # or "tensorflow" or "torch"

import keras_nlp
import keras
import tensorflow as tf

import numpy as np 
import pandas as pd
from tqdm import tqdm
import json

import matplotlib.pyplot as plt
import matplotlib as mpl
import plotly.express as px

In [ ]:
class CFG:
    seed = 42  # Random seed
    preset = "deberta_v3_extra_small_en" # Name of pretrained models
    sequence_length = 512  # Input sequence length
    epochs = 3 # Training epochs
    batch_size = 16  # Batch size
    scheduler = 'cosine'  # Learning rate scheduler
    label2name = {0: 'winner_model_a', 1: 'winner_model_b', 2: 'winner_tie'}
    name2label = {v:k for k, v in label2name.items()}
    class_labels = list(label2name.keys())
    class_names = list(label2name.values())

In [ ]:
keras.utils.set_random_seed(CFG.seed)

In [ ]:
keras.mixed_precision.set_global_policy("mixed_float16")

In [ ]:
BASE_PATH = '/kaggle/input/lmsys-chatbot-arena'

In [ ]:
BASE_PATH

In [ ]:
# Load Train Data
df = pd.read_csv(f'{BASE_PATH}/train.csv') 

# Sample data
# df = df.sample(frac=0.10)

# Take the first prompt and its associated response
df["prompt"] = df.prompt.map(lambda x: eval(x)[0])
df["response_a"] = df.response_a.map(lambda x: eval(x.replace("null","''"))[0])
df["response_b"] = df.response_b.map(lambda x: eval(x.replace("null", "''"))[0])

# Label conversion
df["class_name"] = df[["winner_model_a", "winner_model_b" , "winner_tie"]].idxmax(axis=1)
df["class_label"] = df.class_name.map(CFG.name2label)

# Show Sample
df.head()

In [ ]:
# Load Test Data
test_df = pd.read_csv(f'{BASE_PATH}/test.csv')

# Take the first prompt and response
test_df["prompt"] = test_df.prompt.map(lambda x: eval(x)[0])
test_df["response_a"] = test_df.response_a.map(lambda x: eval(x.replace("null","''"))[0])
test_df["response_b"] = test_df.response_b.map(lambda x: eval(x.replace("null", "''"))[0])

# Show Sample
test_df.head()

In [ ]:
# Define a function to create options based on the prompt and choices
def make_pairs(row):
    row["encode_fail"] = False
    try:
        prompt = row.prompt.encode("utf-8").decode("utf-8")
    except:
        prompt = ""
        row["encode_fail"] = True

    try:
        response_a = row.response_a.encode("utf-8").decode("utf-8")
    except:
        response_a = ""
        row["encode_fail"] = True

    try:
        response_b = row.response_b.encode("utf-8").decode("utf-8")
    except:
        response_b = ""
        row["encode_fail"] = True
        
    row['options'] = [f"Prompt: {prompt}\n\nResponse: {response_a}",  # Response from Model A
                      f"Prompt: {prompt}\n\nResponse: {response_b}"  # Response from Model B
                     ]
    return row

In [ ]:
df = df.apply(make_pairs, axis=1)  # Apply the make_pairs function to each row in df
display(df.head(2))  # Display the first 2 rows of df

test_df = test_df.apply(make_pairs, axis=1)  # Apply the make_pairs function to each row in df
display(test_df.head(2))  # Display the first 2 rows of df

In [ ]:
df.encode_fail.value_counts(normalize=False)

In [ ]:
model_df = pd.concat([df.model_a, df.model_b])
counts = model_df.value_counts().reset_index()
counts.columns = ['LLM', 'Count']

# Create a bar plot with custom styling using Plotly
fig = px.bar(counts, x='LLM', y='Count',
             title='Distribution of LLMs',
             color='Count', color_continuous_scale='viridis')

fig.update_layout(xaxis_tickangle=-45)  # Rotate x-axis labels for better readability

fig.show()

In [ ]:
counts = df['class_name'].value_counts().reset_index()
counts.columns = ['Winner', 'Win Count']

fig = px.bar(counts, x='Winner', y='Win Count',
             title='Winner distribution for Train Data',
             labels={'Winner': 'Winner', 'Win Count': 'Win Count'},
             color='Winner', color_continuous_scale='viridis')

fig.update_layout(xaxis_title="Winner", yaxis_title="Win Count")

fig.show()

In [ ]:
from sklearn.model_selection import train_test_split  # Import package

train_df, valid_df = train_test_split(df, test_size=0.2, stratify=df["class_label"])

In [ ]:
preprocessor = keras_nlp.models.DebertaV3Preprocessor.from_preset(
    preset=CFG.preset, # Name of the model
    sequence_length=CFG.sequence_length, # Max sequence length, will be padded if shorter
)

In [ ]:
outs = preprocessor(df.options.iloc[0])  # Process options for the first row

# Display the shape of each processed output
for k, v in outs.items():
    print(k, ":", v.shape)

In [ ]:
def preprocess_fn(text, label=None):
    text = preprocessor(text)  # Preprocess text
    return (text, label) if label is not None else text  # Return processed text and label if available

In [ ]:
def build_dataset(texts, labels=None, batch_size=32,
                  cache=True, shuffle=1024):
    AUTO = tf.data.AUTOTUNE  # AUTOTUNE option
    slices = (texts,) if labels is None else (texts, keras.utils.to_categorical(labels, num_classes=3))  # Create slices
    ds = tf.data.Dataset.from_tensor_slices(slices)  # Create dataset from slices
    ds = ds.cache() if cache else ds  # Cache dataset if enabled
    ds = ds.map(preprocess_fn, num_parallel_calls=AUTO)  # Map preprocessing function
    opt = tf.data.Options()  # Create dataset options
    if shuffle: 
        ds = ds.shuffle(shuffle, seed=CFG.seed)  # Shuffle dataset if enabled
        opt.experimental_deterministic = False
    ds = ds.with_options(opt)  # Set dataset options
    ds = ds.batch(batch_size, drop_remainder=False)  # Batch dataset
    ds = ds.prefetch(AUTO)  # Prefetch next batch
    return ds  # Return the built dataset

In [ ]:
# Train
train_texts = train_df.options.tolist()  # Extract training texts
train_labels = train_df.class_label.tolist()  # Extract training labels
train_ds = build_dataset(train_texts, train_labels,
                         batch_size=CFG.batch_size,
                         shuffle=True)

# Valid
valid_texts = valid_df.options.tolist()  # Extract validation texts
valid_labels = valid_df.class_label.tolist()  # Extract validation labels
valid_ds = build_dataset(valid_texts, valid_labels,
                         batch_size=CFG.batch_size,
                         shuffle=False)

In [ ]:
import math

def get_lr_callback(batch_size=8, mode='cos', epochs=10, plot=False):
    lr_start, lr_max, lr_min = 1.0e-6, 0.6e-6 * batch_size, 1e-6
    lr_ramp_ep, lr_sus_ep, lr_decay = 2, 0, 0.8

    def lrfn(epoch):  # Learning rate update function
        if epoch < lr_ramp_ep: lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start
        elif epoch < lr_ramp_ep + lr_sus_ep: lr = lr_max
        elif mode == 'exp': lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min
        elif mode == 'step': lr = lr_max * lr_decay**((epoch - lr_ramp_ep - lr_sus_ep) // 2)
        elif mode == 'cos':
            decay_total_epochs, decay_epoch_index = epochs - lr_ramp_ep - lr_sus_ep + 3, epoch - lr_ramp_ep - lr_sus_ep
            phase = math.pi * decay_epoch_index / decay_total_epochs
            lr = (lr_max - lr_min) * 0.5 * (1 + math.cos(phase)) + lr_min
        return lr

    if plot:  # Plot lr curve if plot is True
        plt.figure(figsize=(10, 5))
        plt.plot(np.arange(epochs), [lrfn(epoch) for epoch in np.arange(epochs)], marker='o')
        plt.xlabel('epoch'); plt.ylabel('lr')
        plt.title('LR Scheduler')
        plt.show()

    return keras.callbacks.LearningRateScheduler(lrfn, verbose=False)  # Create lr callback

In [ ]:
lr_cb = get_lr_callback(CFG.batch_size, plot=True)

In [ ]:
ckpt_cb = keras.callbacks.ModelCheckpoint(f'best_model.weights.h5',
                                          monitor='val_log_loss',
                                          save_best_only=True,
                                          save_weights_only=True,
                                          mode='min')

In [ ]:
log_loss = keras.metrics.CategoricalCrossentropy(name="log_loss")

In [ ]:
# Define input layers
inputs = {
    "token_ids": keras.Input(shape=(2, None), dtype=tf.int32, name="token_ids"),
    "padding_mask": keras.Input(shape=(2, None), dtype=tf.int32, name="padding_mask"),
}
# Create a DebertaV3Classifier backbone
backbone = keras_nlp.models.DebertaV3Backbone.from_preset(
    CFG.preset,
)

# Compute embeddings for first response: (P + R_A) using backbone
response_a = {k: v[:, 0, :] for k, v in inputs.items()}
embed_a = backbone(response_a)

# Compute embeddings for second response: (P + R_B), using the same backbone
response_b = {k: v[:, 1, :] for k, v in inputs.items()}
embed_b = backbone(response_b)

# Compute final output
embeds = keras.layers.Concatenate(axis=-1)([embed_a, embed_b])
embeds = keras.layers.GlobalAveragePooling1D()(embeds)
outputs = keras.layers.Dense(3, activation="softmax", name="classifier")(embeds)
model = keras.Model(inputs, outputs)

# Compile the model with optimizer, loss, and metrics
model.compile(
    optimizer=keras.optimizers.Adam(5e-6),
    loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.02),
    metrics=[
        log_loss,
        keras.metrics.CategoricalAccuracy(name="accuracy"),
    ],
)

In [ ]:
model.summary()

In [ ]:
# Start training the model
history = model.fit(
    train_ds,
    epochs=CFG.epochs,
    validation_data=valid_ds,
    callbacks=[lr_cb, ckpt_cb]
)

In [ ]:
model.load_weights('/kaggle/working/best_model.weights.h5')

In [ ]:
# Build test dataset
test_texts = test_df.options.tolist()
test_ds = build_dataset(test_texts,
                         batch_size=min(len(test_df), CFG.batch_size),
                         shuffle=False)

In [ ]:
test_preds = model.predict(test_ds, verbose=1)

In [ ]:
sub_df = test_df[["id"]].copy()
sub_df[CFG.class_names] = test_preds.tolist()
sub_df.to_csv("submission.csv", index=False)
sub_df.head()